###  This script forst sort moves and then outputs it to a move.sort file.

##### Then, it takes the sorted moves and clean them based on the valid vessel data, arrival and sail time. it also generates all the problematic moves.
##### Further, it attaches the data required for ship fouling estimate: duration, ship type, antifouling probability & weight BD for ballast. based on ship type.
##### You can use the moves_cleaned_year.txt for all fruther processes with worring about the preprocessing.
##### Need the temerature and salinity of the ports as well.

In [2]:
from datetime import datetime
import datetime
import time
import collections
from collections import defaultdict
import csv


In [3]:
#Input
year='2005'
if year=='2015' or year=='2018':
    year_range='2015-2018'
    vessel_file='data/moves/vessel_port_info/2015-2018/vessels_'+str(year)+'_sc.csv'
else:
    year_range='1997-2012'
    vessel_file='data/moves/vessel_port_info/'+year_range+'/vsl_'+str(year)+'.lst'

header='VESSEL ID|MOVE REFNO|MOVE ID|PLACE ID|MOVETYPE|MOVETYPE QUALIFIER|ARRIVAL DATE|ARRIVAL DATE ESTIMATED|ARRIVAL DATE QUALIFIER|SAIL DATE|SAIL DATE ESTIMATED|SAIL DATE QUALIFIER|EDIT DATE|STAY DURATION|ROUT|DURATION|VESSEL TYPE|ANTIFOULING PROB|GWT|BALLAST DISCHARGE'
move_file='data/moves/moves_'+str(year)+'.txt'

#Output
move_out='data/moves/moves_cleaned_'+str(year)+'.txt'#clean moves along with fouling factors
move_sorted='data/moves/moves_'+str(year)+'.sort'
print(len(header.split('|')))   

20


## Sorting moves 

In [4]:
# Replacing the arrival time and sail time with datetime objects
#move is sorted by vessel ids
#data clean removes sail=nan or arrival = nan

f=open(move_file,'r') #putting each move in a list 
lines=f.readlines()
f.close()

v_ID=[]
data=[]
for line in lines[1:]:
    l=line.split('|')
    VesselID=l[0].strip()
    arrival=l[6].strip()
    v_ID.append(VesselID)
    data.append(l)

data_clean = list(filter(lambda i: (i[6]!='' and i[9]!=''), data))
move=sorted(data_clean, key=lambda x:int(x[0]))  # sort moves by vessel ids
for m in move:
    arrival=m[6]
    sail=m[9]
    VesselID=m[0]
    try:
        try:
            time_struct_1 = time.strptime(arrival,'%d/%m/%Y %H:%M')
        except:
            time_struct_1 = time.strptime(arrival.split(' ')[0],'%d/%m/%Y')  
        arrival_time=datetime.datetime.fromtimestamp(time.mktime(time_struct_1))
        m[6]= arrival_time

        try:
            time_struct_2 = time.strptime(sail,'%d/%m/%Y %H:%M')
        except:
            time_struct_2 = time.strptime(sail.split(' ')[0],'%d/%m/%Y')  
        sail_time=datetime.datetime.fromtimestamp(time.mktime(time_struct_2))
        m[9]=sail_time
    
    except:
        pass
print(len(data))
print(len(data_clean))

KeyboardInterrupt: 

In [ ]:
#Outputs the two types of key for sorting (sorting for arrival date)
def sorting(L):
    if (len(str(L).split(' '))==1):
        return datetime.datetime.strptime(str(L) , '%Y-%m-%d')
    else:
        return datetime.datetime.strptime(str(L) , '%Y-%m-%d %H:%M:%S')

In [ ]:
# Add all moves corresponding to a ship
vessel_move=defaultdict(list) 
p_vessel_id='0'
for m in move:
    VesselID=m[0]
    if(p_vessel_id==VesselID):
        vessel_move[VesselID].append(m)
    p_vessel_id=VesselID
    
#sort by ship id  
ordered_vessel_move = collections.OrderedDict(sorted(vessel_move.items())) 

In [ ]:
sorted_move=[] #list of all vessel moves .x[6] is the arrival date
for key,item in ordered_vessel_move.items():
    sorted_vessel_move=sorted(ordered_vessel_move[key], key=lambda x: sorting(x[6])) #sorted list of all moves for a vessel 
    sorted_move.append(sorted_vessel_move)

In [11]:
#wrute down the sorted move to a file
with open (move_sorted, 'w') as fp:
    for item in sorted_move:
        for lists in item:
            lists=[str(x).strip() for x in lists]
            t='|'.join(lists)
            fp.write("%s\n" % t)


### cleaning the data and adding the fouling parameters

In [4]:
def GetVesselData(fn, field, delim):
    
    vessel = {}
    with open(fn) as csvfile:
        reader = csv.DictReader(csvfile, delimiter=delim)
        for row in reader:
            vessel[row[field]] = row
    return vessel

In [5]:
categories={} #a translation of vessel types in vessel data file
type_dic={"Auto":["OLC", "PRR", "URC", "URR", "MVE"]
            ,"Container":["UBC", "UCC", "UCR"]
            ,"Tanker":["TAC", "TAS", "TBK", "TCH", "TPD", "LNG", "LNP"
            , "LPG", "TCR", "COO", "CBO", "TCO"]
            ,"Passenger":["MPR", "OFY", "OYT"]
            ,"Bulk":["BBU","BCE","BOR","BWC","UBG"]          
            ,"General":["BCB", "GCT", "GGC","GPC"]          
            ,"Other":["XTG", "OBA", "OTB", "XAH", "XTS", "OSY", "OPO"
            ,"DDR","DTS", "XPT", "DSS", "DTD", "OCL", "OHL", "DHD"]      
            ,"Chemical":["TAC", "TAS", "TBK", "TCH", "TPD"]
            ,"Liquified-Gas":["LNG", "LNP", "LPG"]
            ,"Oil":["TCR", "COO", "CBO", "TCO"]
            ,"Refrigerated-Cargo":["GRF"]
            ,"Fishing":["FFC", "FFF", "FFS", "FTR", "FWH"]
            ,"Research":["RRE"]
            ,"Yacht":["OYT"]
          }
antifouling={'Auto':0.2, 'Container':0.19, 'Tanker':0.3 
             ,'Passenger':0.31, 'Bulk':0.42, 'General':0.53
             ,'Other':0.63, 'Chemical':0.63, 'Liquified-Gas':0.63
             ,'Oil':0.63, 'Refrigerated-Cargo':0.63, 'Fishing':0.63
             , 'Research':0.63, 'Yacht':0.63}
for key,val in type_dic.items():
    for item in val:
        categories[item]=key

### Read vessel files

In [6]:
#read vessel file from sal's and write it to a nice one.
#vessel_file='data/lloyds_all_final_sc.csv' ;vessel_updated='data/lloyds_all_final_updated.txt';other_type_index=-2
vessel_updated='data/moves/vessel_port_info/'+year_range+'/updated_vessels_'+str(year)+'.txt' ;other_type_index=-3
f2 = open(vessel_updated,'w')

vessel_type=defaultdict(list)    
v=open(vessel_file,'r')
if year in ['2018','2015']:
    delim=','
    index=[1,3,7]
    strip_char='"'
else:
    delim='|'
    index=[0,2,5]
    strip_char=''

vdata=v.readlines();v.close()
f2.write(','.join(map(str,[i.strip(strip_char) for i in vdata[0].split('\n')[0].split(',')[1:]])) + '\n')
print(','.join(map(str,[i.strip(strip_char) for i in vdata[0].split('\n')[0].split(',')[1:]])) + '\n')
for ships in vdata[1:]:
    ll=ships.strip().split(delim)
    
    IMO=ll[index[0]+1].strip()
    shipID=ll[index[0]].strip()
    Type=ll[index[1]].strip(strip_char)
    DWT=ll[index[2]].strip()
    
    if Type in categories.keys():
        vessel_type[shipID]=categories[Type]
    else:
        Type_2=ll[other_type_index].strip('"')
        if Type_2=='General Cargo':Type_2='General'
        vessel_type[shipID]= Type_2
            
    f2.write(shipID+','+IMO+','+vessel_type[shipID]+','+','.join(map(str,[i.strip(strip_char) for i in ships.split(delim)][4:])))    

f2.close()

In [7]:
#for years 1997-2012
def get_vessel_type(VesselID,vessel_type,ships_year): #get vessel type from either dictionaries
    mapping={'General Cargo':'General','RoRo':'Auto','Reefer':'Refrigerated-Cargo','Bulker':'Bulk',
            'Tanker ':'Tanker'}

    if VesselID in vessel_type:
        return vessel_type[VesselID]
    elif VesselID in ships_year:
        if ships_year[VesselID]['VESSEL.TYPE'] in categories.keys():
            return categories[ships_year[VesselID]['VESSEL.TYPE']]
        else:
            Type=ships_year[VesselID]['NBIC.code']
            if Type in set(categories.values()):
                return Type
            if Type in mapping.keys():
                return mapping[Type]
            else:
                return -1
    else:
        return -1

In [8]:
o=open(move_out,'w')
o.write(header+'\n')

lines=open(move_sorted,'r').readlines()
print("Number of moves ",len(lines))

i, error=1,0
PrevSail=lines[1].split('|')[9]  
PrevArrival=lines[1].split('|')[6]  

Number of moves  1909589


### For years 1997-2012 use this:

In [9]:
#vessel file containin ballast estimates
ships_year = GetVesselData('data/lloyds_all_final_randomforest.csv', 'VESSEL.ID', ',')  

for line in lines[1:]:
    l=line.strip().split('|')
    if len(l)==14:
        l=l[:-1]
    VesselID , currport =l[0].strip() , l[3].strip()
    pVesselID=(lines[i-1].strip()).split('|')[0].strip() #previous vessel
    prevport=(lines[i-1].strip()).split('|')[3].strip()
    arrival, sail=l[6],l[9]
    VesselType=get_vessel_type(VesselID,vessel_type,ships_year)
    
    if VesselType!=-1 and VesselID in ships_year.keys():
        GWT=ships_year[VesselID]['GROSS']
        ballast_discharge=ships_year[VesselID]['random_forest_br']

        if (PrevSail!='' and PrevArrival!='' and GWT!='NA' and ballast_discharge!='NA' and VesselType!='Combo' and currport!=prevport):
            try:
                time_struct_0=time.strptime(sail,'%Y-%m-%d %H:%M:%S')
                time_struct_1=time.strptime(PrevSail,'%Y-%m-%d %H:%M:%S')
                time_struct_2=time.strptime(arrival,'%Y-%m-%d %H:%M:%S')
                time_struct_3=time.strptime(PrevArrival,'%Y-%m-%d %H:%M:%S')
                
                SailTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_0)) 
                PrevSailTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_1)) #sailtime form prev port to current port, from previous move
                ArrivalTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_2)) #arrival at current port
                PrevArrivalTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_3)) #sail from prev port out
                    
                trip_duration=(ArrivalTime-PrevSailTime).days+round((ArrivalTime-PrevSailTime).seconds//3600/24,2)
                stay_duration=(SailTime-ArrivalTime).days+round((SailTime-ArrivalTime).seconds//3600/24,2) #duration of stay in the source port(prevport)
                antifouling_p=antifouling[VesselType]
                
                if (float(GWT)>=0 and time_struct_2>=time_struct_1 and pVesselID == VesselID):
                    rout=str(stay_duration)+'|'+str(prevport)+'-'+str(currport)+'|'+str(trip_duration)
                    ship_data=str(VesselType)+'|'+str(antifouling_p)+'|'+str(GWT)+'|'+str(ballast_discharge)
                    my_line='|'.join(map(str,l))+'|'+rout+'|'+ship_data+'\n'
                    o.write(str(my_line))
            except:
                error+=1
            
        SrcPort = currport
        PrevSail =sail
        PrevArrival=arrival
        i+=1
    else:
        error+=1

o.close()
print("Number of errors: ",error)

Number of errors:  20994


In [ ]:
Note: for trip durations, some invalid moves are discarded, but they are used for calculating the trip durations.
Stay durations were incorrect. Fixed them.

### For years 2015-2018 use this:

In [10]:
ships = GetVesselData('data/moves/vessel_port_info/2015-2018/updated_vessels_'+str(year)+'.txt', 'VESSEL.ID',',')  
i, error=1,0
PrevSail=lines[1].split('|')[9]  
PrevArrival=lines[1].split('|')[6]  

for line in lines[1:]:
    l=line.strip().split('|')
    if len(l)==14:
        l=l[:-1]
    VesselID , currport=l[0].strip() , l[3].strip()
    pVesselID=(lines[i-1].strip()).split('|')[0].strip() #previous vessel
    prevport=(lines[i-1].strip()).split('|')[3].strip()
    arrival , sail=l[6] , l[9]
    if (PrevSail!='' and PrevArrival!='' and VesselID in ships.keys() and ships[VesselID]['release'] !='NA' and ships[VesselID]['z']!='NA' and currport!=prevport):
        try:
            time_struct_0=time.strptime(sail,'%Y-%m-%d %H:%M:%S')
            time_struct_1=time.strptime(PrevSail,'%Y-%m-%d %H:%M:%S')
            time_struct_2=time.strptime(arrival,'%Y-%m-%d %H:%M:%S')
            time_struct_3=time.strptime(PrevArrival,'%Y-%m-%d %H:%M:%S')

            SailTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_0)) 
            PrevSailTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_1)) #sailtime form prev port to current port, from previous move
            ArrivalTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_2)) #arrival at current port
            PrevArrivalTime=datetime.datetime.fromtimestamp(time.mktime(time_struct_3)) #sail from prev port out
                    
            trip_duration=(ArrivalTime-PrevSailTime).days+round((ArrivalTime-PrevSailTime).seconds//3600/24,2)
            stay_duration=(SailTime-ArrivalTime).days+round((SailTime-ArrivalTime).seconds//3600/24,2) #duration of stay in the source port(prevport)

            ballast_discharge=float(ships[VesselID]['z'])*float(ships[VesselID]['release'])
            GWT=float(ships[VesselID]['GROSS'])
            antifouling_p=antifouling[ships[VesselID]['VESSEL.TYPE']]
            
            if (GWT>=0 and time_struct_2>=time_struct_1 and pVesselID == VesselID):
                rout=str(stay_duration)+'|'+str(prevport)+'-'+str(currport)+'|'+str(trip_duration)
                ship_data=str(ships[VesselID]['VESSEL.TYPE'])+'|'+str(antifouling_p)+'|'+str(GWT)+'|'+str(ballast_discharge)
                my_line='|'.join(map(str,l))+'|'+rout+'|'+ship_data+'\n'
                o.write(str(my_line))
        except:
            error+=1
    else:
        error+=1

    SrcPort = currport
    PrevSail =sail
    PrevArrival=arrival
    i+=1
o.close()
print("Number of errors: ",error)

Number of errors:  1661416
